In [1]:
import pandas as pd
import torch
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as t_d

In [2]:

tracknames = os.listdir("/home/jasper-ubuntu/Documents/Studie/Master AI/Computational intelligence/Torcs-CI/CI/train_data/")
# tracknames = ['aalborg.csv']
datadictin = {}
datadictout = {}
framesin =[]
framesout=[]
for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    datadictin[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 3:]
    datadictout[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 0:3]


#Concatenate all datasets
for i,file in enumerate(tracknames):
    framesin.append(datadictin[i])
    framesout.append(datadictout[i])
    
inp = pd.concat(framesin) 
outp = pd.concat(framesout) 

outp.head()

#Drop missing values
inp.replace('', np.nan, inplace=True)
inp.dropna(inplace=True)
outp.replace('', np.nan, inplace=True)
outp.dropna(inplace=True)
outp.head()

train_data/road_alpine-1_7cars_2475231.csv
train_data/road_alpine-1_7cars_2475119.csv
train_data/road_alpine-1_7cars_2475043.csv
train_data/road_alpine-1_7cars_2475155.csv


,accel,brake,steer
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [4]:
inp_size = inp.shape[1]
outp_size = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,inp_size,outp_size)

132708 72 3


In [4]:
# # Get data from pandas in a Tensor Variable
# x = Variable(torch.from_numpy(inp.iloc[:,:].as_matrix()).float())
# y = Variable(torch.from_numpy(outp.iloc[:,:].as_matrix()).float())
# print(x)
# print(y)

Variable containing:
     0.0000     -0.9820   6306.6499  ...     200.0000    200.0000    200.0000
     0.0000     -0.9620   6306.6499  ...     200.0000    200.0000    200.0000
     0.0000     -0.9420   6306.6499  ...     200.0000    200.0000    200.0000
                ...                   ⋱                   ...                
    -0.0359     90.9260   2000.6600  ...     200.0000    200.0000    200.0000
    -0.0366     90.9480   2000.9600  ...     200.0000    200.0000    200.0000
    -0.0363     90.9700   2001.2700  ...     200.0000    200.0000    200.0000
[torch.FloatTensor of size 132708x72]

Variable containing:
 1.0000  0.0000  0.0000
 1.0000  0.0000  0.0000
 1.0000  0.0000  0.0000
           ⋮            
 1.0000  0.0000 -0.0226
 1.0000  0.0000 -0.0759
 1.0000  0.0000 -0.0557
[torch.FloatTensor of size 132708x3]



In [11]:
#Neural Net Classes

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, outp_size)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out


    
    
    
class SimpleRNN(torch.nn.Module):
    def __init__(self, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size

        self.inp = torch.nn.Linear(25, hidden_size)
        self.rnn = torch.nn.LSTM(hidden_size, hidden_size, 2, dropout=0.05)
        self.out = torch.nn.Linear(hidden_size, 3)

    def step(self, input, hidden=None):
        #print(input.view(1, -1).unsqueeze(1))
        input = self.inp(input.view(1, -1)).unsqueeze(1)
        output, hidden = self.rnn(input, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

    def forward(self, inputs, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(inputs)
        outputs = Variable(torch.zeros(steps, 3, 1))
        for i in range(steps):
            if force or i == 0:
                input = inputs[i]
            else:
                input = output
            output, hidden = self.step(input, hidden)
            outputs[i] = output
        return outputs, hidden



In [15]:
#Hyper params
sequence_length = 1
input_size = inp.shape[1]
hidden_size = 50
num_layers = 3
output_size = outp.shape[1]
batch_size = 100
num_epochs = 10
learning_rate = 0.001


# Construct dataset variables
x = torch.from_numpy(inp.iloc[:,:].as_matrix()).float()
y = torch.from_numpy(outp.iloc[:,:].as_matrix()).float()

data_set = torch.utils.data.TensorDataset(x, y)
train_loader = torch.utils.data.DataLoader(dataset=data_set,batch_size=batch_size,shuffle=True)


# Initialize model instance

model = LSTM(input_size, hidden_size, output_size, batch_size, num_layers)

# Train the Model


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = LSTM(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, i+1, len(sensors), loss.data[0]))

TypeError: __init__() missing 4 required positional arguments: 'hidden_size', 'output_size', 'batch_size', and 'num_layers'

In [16]:
import torch

from collections import defaultdict
import time
import random
import torch
from torch.autograd import Variable
import torch.autograd
from random import shuffle
import torch.nn as nn
import numpy as np
import time, math
import sys
import pandas as pd

class simpleGRU(nn.Module):
    def __init__(self, D_in, h_layer_size, n_hidden_layers, D_out):
        super(simpleGRU, self).__init__()
        self.D_in = D_in
        self.h_layer_size = h_layer_size
        self.D_out = D_out
        self.n_hidden_layers = n_hidden_layers

        self.encoder = nn.Linear(D_in, h_layer_size)
        # self.encoder = nn.Embedding(D_in, h_layer_size)
        self.gru = nn.GRU(h_layer_size, h_layer_size, n_hidden_layers)
        self.decoder = nn.Linear(h_layer_size, D_out)
    
    def forward(self, input, hidden):

        encoded = self.encoder(input.view(1, -1))
        out, hidden = self.gru(encoded.view(1, 1, -1), hidden)
        output = self.decoder(out.view(1, -1))

        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_hidden_layers, 1, self.h_layer_size))


def random_train(seq_length):
    start_ix = random.randint(0, data_size - seq_length)
    end_ix = start_ix + seq_length + 1
    sequence = data[start_ix:end_ix]
    
    inp = char_tensor(sequence[:-1])
    target = char_tensor(sequence[1:])
    return inp, target

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(seq_length):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c][0:3].view(1, -1))

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / seq_length

def read_data(filename):
    pd_data = pd.read_csv('all_tracks.csv')
    data = torch.from_numpy(pd_data.values).type(torch.FloatTensor)
    data_size = data.shape[0]
    D_in = data.shape[1]
    return data, data_size, D_in

def get_train_pair(data, start_ix, seq_length):
    end_ix = start_ix + seq_length + 1
    sequence = data[start_ix:end_ix]

    inp = Variable(sequence[:-1])
    target = Variable(sequence[1:])

    return inp, target



if __name__ == '__main__':
    np.random.seed(1)

    seq_length = 25 # number of steps to unroll the RNN for
    hidden_size = 25
    n_hidden_layers = 3
    D_out = 3
    lr = 0.005

    n_epochs = 100
    print_every = 1
    plot_every = 1
    save_every = 1

    # Read in file
    filename = 'all_tracks.csv'
    data, data_size, D_in = read_data(filename)
    print('Data length: %d' % data_size)

    # Setup network
    decoder = simpleGRU(D_in, hidden_size, n_hidden_layers, D_out)
    # decoder.load_state_dict(torch.load('simpleGRU_epoch_2000file_sherlock.txt.pkl'))

    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    criterion = nn.L1Loss()

    start = time.time()
    all_losses = []
    loss_avg = 0

    print('Starting training: Total Epochs: %d' % (n_epochs))
    # Start training
    for epoch in range(n_epochs):        
        counter = 0
        # Create random indexes for training sequences
        total_samples = data_size // (seq_length + 1)
        sample_list = list(range(total_samples))
        shuffle(sample_list)

        for ix in sample_list:
            inp, target = get_train_pair(data, ix, seq_length)
            # print(inp, target)
            # sys.exit()

            loss = train(inp, target)
            loss_avg += loss
            counter += 1
            if counter % 500 == 0:
                print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))


        if epoch % print_every == 0:
            print('[Finished Epoch %s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            # print(evaluate('I ', predict_len=100, temperature=0.4), '\n')
            

        if epoch % plot_every == 0:
            all_losses.append(loss_avg / plot_every)
            loss_avg = 0
        
        if epoch % save_every == 0:
            torch.save(decoder.state_dict(), 'simpleGRU_epoch_' + str(epoch) + '_' + filename + '.pkl')
            print("Model saved, epoch: %d" % (epoch))

    f= open("all_losses.txt","w+")
    for i in range(len(all_losses)):
        f.write(str(all_losses[i]) + ',')


FileNotFoundError: File b'all_tracks.csv' does not exist